<a href="https://colab.research.google.com/github/NMashalov/ODE-Numerical-Solutions/blob/main/Shallow_water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инсталляция программных пакетов

In [ ]:
!apt-get install vtk6
!apt-get install python-vtk
!pip install numpy vtk pyqt5
! pip install scipy jupyter ipywidgets ipyevents



Reading package lists... Done
Building dependency tree       
Reading state information... Done
vtk6 is already the newest version (6.3.0+dfsg1-11build1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package python-vtk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'python-vtk' has no installation candidate


In [ ]:
!pip install mayavi

In [ ]:
 !apt-get install -qq xvfb
 import os     #For interacting with the Operating System
 os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
 #Define environmental variable using os.environ object
 os.environ['DISPLAY'] = ':99'
 import panel as pn
 pn.extension('vtk') #for interacting with complex 3D geometry

In [ ]:
 import numpy as np
 from mayavi import mlab

In [ ]:
mlab.init_notebook()

Notebook initialized with ipy backend.


# Решение уравнений малой воды схемой Лакса-Вендроффа

In [ ]:
!rm -rf 3d_image
!mkdir 3d_image

!rm -rf Fourier_image
!mkdir Fourier_image

In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import RectBivariateSpline

from matplotlib.colors import LightSource
from mayavi import mlab
import sys


import matplotlib.pyplot as plt, mpl_toolkits.mplot3d
import os

n = 50

x_size=1
y_size=1.

dx = x_size / n
dy = y_size / n
# сетка для целых узлов
x = np.linspace(-dx / 2, x_size + dx / 2, n + 2)
y = np.linspace(-dy / 2, y_size + dy / 2, n + 2)

dx = x[1] - x[0]
dy = y[1] - y[0]

dt = 0.001

# h,u,v - величина
h = 5*np.ones((n + 2, n + 2))

U = np.zeros((n + 2, n + 2))
V = np.zeros((n + 2, n + 2))

# Начальное возмущение
xx, yy = np.meshgrid(x, y)

x_0 = x_size/2
y_0 = y_size/2

alpha = 100
h += 0.01 * np.exp(-alpha * ((xx - x_0) ** 2 + (yy - y_0) ** 2))



# Полуцелые узлы для сетки по x
h_x = np.zeros((n + 1, n + 1))
U_x = np.zeros((n + 1, n + 1))
V_x = np.zeros((n + 1, n + 1))

# Полуцелые узлы для сетки по у
h_y = np.zeros((n + 1, n + 1))
U_y = np.zeros((n + 1, n + 1))
V_y = np.zeros((n + 1, n + 1))

# Число итераций по времени

s=500

resourse=np.empty((s,n,n))

for i in range(s):
    resourse[i] = h[1:-1, 1:-1]


    # Выполняем условия равенства потока 0 на границах

    # равенство объёмов
    h[:, 0] = h[:, 1]
    h[:, -1] = h[:, -2]
    h[0, :] = h[1, :]
    h[-1, :] = h[-2, :]

    # равенство скоростей по x
    U[0, :] = -U[1, :]
    U[-1, :] = -U[-2, :]

    # равенство скоростей по y
    V[:, 0] = -V[:, 1]
    V[:, -1] = -V[:, -2]

    # начинаем c направления по x
    h_x = 1 / 2 * (h[1:, 1:] + h[:-1, 1:]) - dt / (2 * dx) * (U[1:, 1:] - U[:-1, 1:])
    U_x = 1 / 2 * (U[1:, 1:] + U[:-1, 1:]) - dt / (2 * dx) * (
                U[1:, 1:] ** 2 / h[1:, 1:] - U[:-1, 1:] ** 2 / h[:-1, 1:] + 4.4 * (h[1:, 1:] ** 2 - h[:-1, 1:] ** 2))
    V_x = 1 / 2 * (V[1:, 1:] + V[:-1, 1:]) - dt / (2 * dx) * (
                U[1:, 1:] * V[1:, 1:] / h[1:, 1:] - U[:-1, 1:] * V[:-1, 1:] / h[:-1, 1:])

    #print(h_x)

    # начинаем c направления по y
    h_y = 1 / 2 * (h[1:, 1:] + h[1:, :-1]) - dt / (2 * dy) * (V[1:, 1:] - V[1:, :-1])
    U_y = 1 / 2 * (U[1:, 1:] + U[1:, :-1]) - dt / (2 * dy) * (
                U[1:, 1:] * V[1:, 1:] / h[1:, 1:] - U[1:, :-1] * V[1:, :-1] / h[1:, :-1])
    V_y = 1 / 2 * (V[1:, 1:] + V[1:, :-1]) - dt / (2 * dy) * (
                V[1:, 1:] ** 2 / h[1:, 1:] - V[1:, :-1] ** 2 / h[1:, :-1] + 4.4 * (h[1:, 1:] ** 2 - h[1:, :-1] ** 2))

    #финальный шаг

    h[1:-1, 1:-1] += -dt / dx * (U_x[1:, :-1] - U_x[:-1, :-1]) - dt / dy * (V_y[:-1, 1:] - V_y[:-1, :-1])
    U[1:-1, 1:-1] += -dt / dx * (U_x[1:, :-1] ** 2 / h_x[1:, :-1] - U_x[:-1, :-1] ** 2 / h_x[:-1, :-1] + 4.4 * (h_x[1:, :-1] ** 2 - h_x[:-1, :-1] ** 2)) - dt / dy * (U_y[:-1, 1:] * V_y[:-1, 1:] / h_y[:-1, 1:] - U_y[:-1, :-1] * V_y[:-1, :-1] / h_y[:-1, :-1])
    V[1:-1, 1:-1] += -dt / dx * (U_x[:-1, 1:] * V_x[:-1, 1:] / h_x[:-1, 1:] - U_x[:-1, :-1] * V_x[:-1, :-1] / h_x[:-1,:-1]) - dt / dy * (U_y[:-1, 1:] ** 2 / h_y[:-1, 1:] - U_y[:-1, :-1] ** 2 / h_y[:-1, :-1] + 4.4 * (h_y[:-1, 1:] ** 2 - h_y[:-1, :-1] ** 2))



x = np.linspace(dx / 2, x_size - dx / 2, n)
y = np.linspace(dy / 2, y_size - dy / 2, n )
xx, yy = np.meshgrid(x, y)
fig, ax = plt.subplots(1, 1, figsize=(6, 9))

fft_resourse=np.log10(np.abs(np.fft.fftshift(np.fft.fft2(resourse-5),axes=(-2,-1))))

for i in range(s):
  znow = fft_resourse[i]
  im=ax.imshow(znow,cmap='Greys')
  plt.savefig("Fourier_image/ani_{0:03d}.png".format(i))
  ax.clear()

plt.close()

a=640*2
b=480*2


fig = mlab.figure(size=(a, b), bgcolor=(1, 1, 0.9))
surf = mlab.surf(x.T, y.T, resourse[0, :, :].T, colormap='ocean',warp_scale='auto',vmin=4.99)
# Change the visualization parameters.
surf.actor.property.interpolation = 'phong'
surf.actor.property.specular = 0.1
surf.actor.property.specular_power = 100
surf.actor.property.ambient = 0
mlab.view(azimuth=235, elevation=60,distance=150)#,focalpoint=(0.35 , 0.25 , 0.0))

for i in range(s):
    znow = resourse[i, :,:].T  # eta_polar( rnow, tnow )
    surf.mlab_source.scalars = znow
    mlab.savefig("3d_image/ani_{0:03d}.png".format(i))



# Анимация

3D изображение возмущения

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os
os.system(f"ffmpeg -f image2 -r 24 -i 3d_image/ani_%03d.png -vcodec libx264 -y video.mp4")

# Show video
mp4 = open("video.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

2D Фурье-образ возмущения

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os
os.system(f"ffmpeg -f image2 -r 24 -i Fourier_image/ani_%03d.png -vcodec libx264 -y video.mp4")

# Show video
mp4 = open("video.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)